## Installing neccessary packages

In [1]:
pip install google-cloud-bigquery

Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for proto-plus: filename=proto_plus-1.11.0-py3-none-any.whl size=41575 sha256=f56dc4a7728af80baf3312eba482006fbdc66b2fd63f7dbf5d8e708654fbab99
  Stored in directory: c:\users\elais\appdata\local\pip\cache\wheels\c3\37\d7\f09b314e0099f7c14b1fb52084680544260470cc3bc20a82ed
  Created wheel for google-crc32c: filename=google_crc32c-1.0.0-py3-none-any.whl size=9922 sha256=453ef1f6255c3177be1ae6cd16aaac5badde7f029dc041a0548929450c122064
  Stored in directory: c:\users\elais\appdata\local\pip\cache\whe

In [2]:
pip install --upgrade google-auth

Requirement already up-to-date: google-auth in c:\users\elais\anaconda3\lib\site-packages (1.23.0)
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install pandas_gbq

Note: you may need to restart the kernel to use updated packages.


## Connecting to the cloud using a json file that I downloaded for authentication:

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('C:/Users/elais/CIS4400-Group-3d11c40a3827.json')
project_id = 'cis4400-group'
client = bigquery.Client(credentials=credentials,project=project_id)

print(client)

## Creating the dataset on GBQ:

In [3]:
dataset_ref = client.dataset('NBA_All_Decade')

dataset_object = bigquery.Dataset(dataset_ref)
gbq_dataset = client.create_dataset(dataset_object)
print('Dataset {} created.'.format(gbq_dataset.dataset_id))

Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/cis4400-group/datasets?prettyPrint=false: Already Exists: Dataset cis4400-group:NBA_All_Decade

In [4]:
print(dataset_ref)

DatasetReference('cis4400-group', 'NBA_All_Decade')


## Creating pandas DFs to match our relational model and loading the data to gbq:

In [7]:
# importing necessary modules

from google.cloud import bigquery
import pandas as pd

In [57]:
# Reading csv files and creating dataframes to load to my data warehouse

matchesDim = pd.read_csv('NBA_Games_all_Decade - NBA_Games_all_Decade.csv')
matchesDim['Date'] = pd.to_datetime(matchesDim.Date) # Changing date to be datetime type so we can analyze by dates
playerDim = pd.read_csv('Players_3D.csv')
DateDim = pd.DataFrame(matchesDim['Date'])
DateDim['Date'] = pd.to_datetime(DateDim.Date)
DateDim['Month'] = pd.DatetimeIndex(DateDim['Date']).month # adding broken-down date columns for further analysis
DateDim['DayOfMonth'] = pd.DatetimeIndex(DateDim['Date']).day
DateDim['Year'] = pd.DatetimeIndex(DateDim['Date']).year
DateDim['DayOfWeek'] = DateDim['Date'].dt.day_name()

In [59]:
# Loading all dataframes to my data warehouse as tables

matchesDim.to_gbq('Decade.MatchesDimension', project_id='cis4400-group')
playerDim.to_gbq('NBA_All_DNBA_All_ecade.PlayersDimension', project_id='cis4400-group')
DateDim.to_gbq('NBA_All_Decade.DatesDimension', project_id='cis4400-group')

1it [00:09,  9.89s/it]
